In [1]:
URL = "http://covidwizard.com"

import pandas as pd
import logging

from bs4 import BeautifulSoup
from selenium import webdriver
from time import sleep
from tqdm import tqdm

from selenium.webdriver.support.ui import Select
from src.utils import COUNTRIES

In [14]:
def load_html(country):
    try:
        op = webdriver.ChromeOptions()
        op.add_argument('headless')
        driver = webdriver.Chrome(options=op)
        driver.get(URL)

        selector = Select(driver.find_element_by_id("country"))
        driver.find_element_by_id('country').click()
        selector.select_by_visible_text(COUNTRIES[-1])

        driver.find_element_by_id('open_table').click()
        driver.switch_to.active_element
        sleep(5)

        table = driver.find_element_by_id("data_table").get_attribute("innerHTML")
        driver.close()
        
        return table
    except Exception as e:
        logging.warning(ValueError(f"Can't load data for {country}"))
        
        
def parse_html(table):
    try:
        soup = BeautifulSoup(table)
        header = [x.text for x in soup.find_all("table")[0].find("tr")]
        data = []
        for row in soup.find_all("table")[0].find_all("tr")[1:]:
            row = [x.text for x in row]
            assert len(row) == len(header), row
            data.append(row)

        df = pd.DataFrame(data, columns=header)
        df.Date = pd.to_datetime(df.Date)
        df[header[1:]] = df[header[1:]].astype(float)
        df[header[1:-3]] = df[header[1:-3]].astype(int)
        
        return df.set_index("Date")
        
    except Exception as e:
        print(e)
        logging.warning(ValueError(f"Can't parse data"))

In [ ]:
res = {}

for country in tqdm(COUNTRIES):
    table = load_html(country)
    df = parse_html(table)
    res[country] = df
    df.to_csv(f"data/{country}.csv")


  0%|          | 0/9 [00:00<?, ?it/s]